In [49]:
import sys
print(sys.executable)

/Users/huanxing/Documents/GitHub/Analogical-LLM/analogical-lc-env/bin/python


In [77]:
import getpass
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import openai 
# from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model
from textwrap import dedent
import json

In [51]:
# Prompts
TARGET_DOMAIN = dedent("""
        As a Domain Analysis Specialist, extract all the core innovation domains from the user query. It could be a single one for a simple query, or multiple ones for a complex query.
        Instructions:
        1. Analyze the user's input
        2. Identify the primary domain(s) requiring innovation
        3. Classify it within standard innovation categories
        Output Format:
        Target Domain: [Clear, specific domain label]
        Be very detailed and specific in your response and do not generalize. Respond ONLY with the name of the domain, do NOT include ANY other text like 'Target Domain:'.
        User query: {user_query}
""").strip()

PROBLEM_LANDSCAPE = dedent("""
        You are a Problem Landscape Analyst. Your task is to map out the concrete challenges within the target domain identified.
        Instructions:
        1. Identify all the core problems or challenges currently present in these domains. Aim for at least 3 problems per domain
        2. For each problem, provide:
        - Problem: A short, clear title.
        - Description: 2-3 sentences explaining what the problem is and why it matters.
        - Context: Briefly state the circumstances or environment where this problem occurs.
        - Stakeholders: List the main groups or individuals affected.
        - Root Causes: Identify 1-3 underlying causes, if known.
        - Impact: State the significance of the problem (e.g., social, economic, technical).
        - Current Approaches: How is this problem currently addressed?
        - Limitations: What are the shortcomings of current approaches?
        - Success Metrics: How would you measure if this problem is solved?
        - Interconnections: Note if this problem is linked to or influenced by other problems.
        Output Format:
        Present your findings as a structured list or JSON array, with each problem fully described as above.
        Important:
        - Focus on clarity and completeness.
        - Avoid abstracting or generalizing; stay concrete and domain-specific.
        - Do not propose solutions; only describe the current problem landscape.
        Target domain: {target_domain}
""").strip()

ABSTRACTION = dedent("""
You are a TRIZ Methodology Expert. Transform domain-specific problems into universal contradictions.
        Process:
        1. Read up on TRIZ - the contradiction matrix, and the inventive principles
        2. For each problem in the problem landscape:
        - Abstract to universal parameters (what improves vs. what worsens)
        - Express as 'When we improve X, Y worsens'
        - Ensure parameters are domain-agnostic
        3. Analyze all the abstracted universal parameters, and identify all the core TRIZ contradictions present:
        - Select the most fundamental tensions
        - Map to TRIZ contradiction matrix
        - Note applicable inventive principles
        Output:
        # List all the core contradictions in form of:
        - Improving [parameter] vs. Worsening [parameter]
        - TRIZ Principles: [1-3 relevant principles]
        - Innovation Potential: [High/Medium/Low]
        Focus on contradictions that, if resolved, would create breakthrough value.

        Problem landscape: {problem_landscape}
""").strip()

BASE_DOMAIN = dedent("""
        You are a Cross-Domain Search Specialist. Do the following:
        - For each contradiction provided, identify 3 distinct source domains (fields or industries) where this contradiction has been successfully addressed.
        - Experiment with different subsets of the list of contradictions, and see if you could identify 3 distinct source domains for each of these subsets identified as well. You should find at least 3 different subsets.
        Note: The domains should have A CONCEPTUAL DISTANCE OF AT LEAST 3 DISTINCT HOPS FROM WHAT IMMEDIATELY COMES TO MIND. Be creative! It can be domains within spheres like natural, phsyical, social, artistic, or anything.
        For each domain identified, briefly explain why it is relevant to the single contradiction or the subset of contradictions identified. Do not describe specific solutions just yet-only list the domains and your rationale.
        Output:
        A list for each contradiction and subset of contradictions identified, naming 3 relevant domains with a 2 sentence rationale for each.
        Aim for a total of at least 20 relevant base domains. 
        Contradictions: {contradictions}
""").strip()

BASE_SOLUTIONS = dedent("""
        You are a Solution Pattern Extractor. You are provided with an input with 3 base domains identified per TRIZ (Theory of Inventive Problem Solving) contradiction or a set of contradictions, as well as the contradictions themselves.
        For each of these identified base domains, identify one specific, well-documented solution pattern within the domain that effectively resolves the contradiction (or the set of contradictions).
        For each solution pattern, return:
        - Identify the base domain it's corresponding to
        - Recall the contradiction or the set of contradictions that this base domain faces
        - The name or label of the solution pattern for resolving these contradiction(s) in the base domain
        - A detailed description of the core mechanism or principle involved and how it addressed the domain's contradiction(s)
        - The context or situation in the domain where this pattern is applied\n"
        Do not generalize or adapt the solution-simply describe how the contradiction is addressed within each source domain.
        Output:
        For each of the provided domain, list the base domain name, contradiction(s) faced, solution pattern name, the detailed description of the mechanism of the solution pattern, and the context in which it is used. Articulate the contradictions as problems and considerations faced, through framing them as a tension.

        Input: {input}
""").strip()

ANALOGICAL_TRANSFER = dedent("""
        You are a very innovative Analogical Transfer Specialist.
        You are provided with list the base domain name, tensions faced, solution pattern name, the detailed description of the mechanism of the solution pattern, and the context in which it is used.
        Your task is to propose how solution patterns used to resolve these tensions in various base domains might inspire solution framings for the original target domain.

        Input Overview:
        1. A list the base domains identified, the tensions these domains faced, the name of solution patterns that helped addressed these tensions in these base domains, the detailed description of the mechanism of the solution pattern, and the context in which it is used.
        2. The original target domain.

        Instructions:
        For each pair of base domain and the corresponding tensions identified, review the solution patterns that worked for the base domain. For each pattern:
        - Analyze the core mechanism or principle behind the solution.
        - Map and adapt this mechanism conceptually to the target domain, considering the specific context and needs of the target domain.
        - Clearly describe how this analogical transfer could frame a potential solution in the target domain.
        - Highlight any key adaptations, considerations, or limitations that would be relevant when applying this pattern to the target domain.

        Your expected Output:
        For each base domain, provide a comprehensive description of a proposed solution framing for the target domain, including:
        - The original tension addressed
        - The source domain and solution pattern
        - A detailed explanation of how the pattern could inspire or inform a solution in the target domain
        - Any important adaptations or considerations for successful transfer

        Here are the actualinputs:
        - A list the base domains identified, the tensions these domains faced, the name of solution patterns that helped addressed these tensions in these base domains, the detailed description of the mechanism of the solution pattern, and the context in which it is used.: {contradictions_solutions}
        - Original target domain: {target_domain}
""").strip()

In [52]:
# check for openai API
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")

In [53]:
# Graph state: workflow
class ReasoningState(TypedDict):
    user_query: str
    target_domain: str
    problem_landscape: str
    abstraction: str
    abstraction_feedback: str
    base_domain: str
    base_solutions: str
    analogical_transfer: str
    transfer_feedback: str
    solution: str

In [54]:
# Can switch between different LLMs 
llms = init_chat_model("openai:gpt-4.1")

In [55]:
# 1st Agent: Identify Target Domain from User Input
def target_domain_agent(state: ReasoningState):
    u = state['user_query']

    msg = llms.invoke(TARGET_DOMAIN.format(user_query=u))

    return {"target_domain": msg.content}

In [56]:
# 2st Agent: Conduct comprehensive research into problem landscape for target domain 
# (i.e. What specific challenges exist in this domain?)
def problem_landscape_agent(state: ReasoningState):
    t = state['target_domain']

    msg = llms.invoke(PROBLEM_LANDSCAPE.format(target_domain=t))

    return {"problem_landscape": msg.content}

In [57]:
# 3rd Agent: Abstract Problems Identified into Generalized Principles and TRIZ Contradiction
def abstraction_agent(state: ReasoningState):
    p = state['problem_landscape']
    feedback = state.get('abstraction_feedback', '')
    
    # If there's feedback, include it in the prompt
    if feedback:
        msg = llms.invoke(ABSTRACTION.format(
            problem_landscape=p,
            feedback=f"\nPrevious attempt feedback: {feedback}\nPlease address these issues in your new abstraction."
        ))
    else:
        msg = llms.invoke(ABSTRACTION.format(problem_landscape=p))
    
    return {"abstraction": msg.content}

In [78]:
## 3.5th agent: CRIT Quality Control for TRIZ Abstraction
def CRIT_Control_Abstraction(target_domain: str, problem_landscape: str, triz_contradictions: str) -> dict:
    """Quality control function that evaluates the abstraction output."""
    prompt = f"""
        # CRITICAL READING INQUISITIVE TEMPLATE (CRIT) - TRIZ VALIDATION
        You are a TRIZ ABSTRACTION VALIDATOR. Apply Socratic methods to evaluate contradiction quality.

        ## INPUTS
        Target Domain: {target_domain}
        Problem Landscape: {problem_landscape}
        TRIZ Contradictions: {triz_contradictions}

        ## VALIDATION PROTOCOL [Pass Threshold: 7.0/10]

        ### 1. METHOD OF DEFINITION
        - Does each contradiction follow "Improving [X] vs. Worsening [Y]" format?
        - Are parameters from Altshuller's 39 Engineering Parameters?
        - Is abstraction at first-principles level (not solution-biased)?

        ### 2. METHOD OF ELENCHUS (Cross-Examination)
        - Trace each contradiction back to originating problem
        - Verify causal chain: Problem → Root Cause → Contradiction
        - Rate abstraction mapping strength (1-10)

        ### 3. METHOD OF DIALECTIC (Counter-Arguments)
        - Generate 2-3 alternative contradictions for same problem
        - Compare which better captures essential tension
        - Eliminate weaker contradictions

        ### 4. SCORING CRITERIA
        - **TRIZ Compliance (40%):** Format, parameter accuracy
        - **Abstraction Quality (35%):** First-principles, non-solution-biased
        - **Problem Coverage (25%):** All major problems represented

        ### OUTPUT FORMAT
        {{
            "score": float (1.0-10.0),
            "pass": boolean,
            "reason": str (brief explanation if fail)
        }}

        **FAILURE CONDITIONS:** Format violations, non-TRIZ parameters, solution bias, logical gaps, incomplete coverage.
        
        IMPORTANT: Your response must be a valid JSON object with the exact keys shown above.
    """
    response = llms.invoke(prompt)
    
    # Parse the response into a dictionary
    try:
        # Extract JSON from the response
        response_text = response.content.strip()
        # Find the JSON object in the response
        start_idx = response_text.find('{')
        end_idx = response_text.rfind('}') + 1
        if start_idx >= 0 and end_idx > start_idx:
            json_str = response_text[start_idx:end_idx]
            result = json.loads(json_str)
            return result
        else:
            # If no JSON found, return a default fail response
            return {
                "score": 0.0,
                "pass": False,
                "reason": "Invalid response format from validator"
            }
    except json.JSONDecodeError:
        # If JSON parsing fails, return a default fail response
        return {
            "score": 0.0,
            "pass": False,
            "reason": "Failed to parse validator response"
        }

In [59]:
## 3.5th Agent Logic Flow: if quality control passes, continue to next agent. Else, loop back to retry TRIZ abstraction (taking rationale for why it failed as additional input)
def should_continue_to_base(state: ReasoningState) -> str:
    """Determine whether to continue to base domain or retry abstraction based on CRIT score."""
    # Get the CRIT score from the state
    crit_result = CRIT_Control_Abstraction(
        state['target_domain'],
        state['problem_landscape'],
        state['abstraction']
    )
    
    # Parse the result to get the score and pass status
    score = crit_result.get('score', 0)
    passed = crit_result.get('pass', False)
    
    if passed:
        return "base"  # Continue to base domain agent
    else:
        # Add the feedback to the state for the abstraction agent to use
        state['abstraction_feedback'] = crit_result.get('reason', '')
        return "abstract"  # Retry abstraction agent


In [60]:
# 4th Agent: Search for Appropriate Base Domains
def base_domain_agent(state: ReasoningState):
    a = state['abstraction']

    msg = llms.invoke(BASE_DOMAIN.format(contradictions=a))

    return {"base_domain": msg.content}

In [61]:
# 5th Agent: Identify Solution in Base Domain
def base_solution_agent(state: ReasoningState):
    b = state['base_domain']

    msg = llms.invoke(BASE_SOLUTIONS.format(input=b))

    return {"base_solutions": msg.content}

In [62]:
# 6th Agent: Base Domain Solution Informing Target Domain Solution
def analogical_transfer_agent(state: ReasoningState):
    if "base_solutions" not in state:
        raise ValueError("Missing 'base_solutions' key. Check if previous node returned it.")
    b = state['base_solutions']
    t = state['target_domain']
    feedback = state.get('transfer_feedback', '')
    
    # If there's feedback, include it in the prompt
    if feedback:
        msg = llms.invoke(ANALOGICAL_TRANSFER.format(
            contradictions_solutions=b,
            target_domain=t,
            feedback=f"\nPrevious attempt feedback: {feedback}\nPlease address these issues in your new analogical transfer."
        ))
    else:
        msg = llms.invoke(ANALOGICAL_TRANSFER.format(
            contradictions_solutions=b,
            target_domain=t
        ))
    
    return {"analogical_transfer": msg.content}

In [79]:
# 6.5th Agent: CRIT Quality Control for Analogical Reasoning 
def CRIT_Control_Analogical_Transfer(base_solutions: str, analogical_transfers: str, original_contradictions: str, target_domain: str) -> dict:
    prompt = f"""
    # CRITICAL READING INQUISITIVE TEMPLATE (CRIT) - ANALOGICAL TRANSFER VALIDATION
    You are an ANALOGICAL TRANSFER VALIDATOR. Apply Socratic methods to evaluate transfer quality.

    ## INPUTS
    Base Solutions: {base_solutions}
    Analogical Transfers: {analogical_transfers}
    Original Contradictions: {original_contradictions}
    Target Domain: {target_domain}

    ## VALIDATION PROTOCOL [Pass Threshold: 7.5/10]

    ### 1. METHOD OF ELENCHUS (Cross-Examination)
    - Does each transfer preserve the CORE MECHANISM from base domain?
    - Trace: Base Solution → Transfer Logic → Target Application
    - Rate mechanism preservation fidelity (1-10)

    ### 2. METHOD OF DIALECTIC (Counter-Arguments)
    - Generate 2-3 alternative transfer approaches for same base solution
    - Identify potential failure modes in target domain
    - Test against target domain constraints

    ### 3. METHOD OF COUNTERFACTUAL
    - "What if this transfer were applied in [alternative context]?"
    - Predict 2-3 unintended consequences in target domain
    - Validate scalability and implementation feasibility

    ### 4. SCORING CRITERIA
    - **Mechanism Fidelity (35%):** Core solution pattern preserved
    - **Domain Adaptation (30%):** Properly adapted to target constraints  
    - **Contradiction Resolution (25%):** Addresses original TRIZ tensions
    - **Implementation Feasibility (10%):** Realistic in target context

    ### OUTPUT FORMAT
    {{
        "score": float (1.0-10.0),
        "pass": boolean,
        "reason": str (brief explanation if fail)
    }}

    **FAILURE CONDITIONS:** Mechanism distortion, poor domain adaptation, contradiction mismatch, implementation impossibility.
    
    IMPORTANT: Your response must be a valid JSON object with the exact keys shown above.
    """
    response = llms.invoke(prompt)
    
    # Parse the response into a dictionary
    try:
        # Extract JSON from the response
        response_text = response.content.strip()
        # Find the JSON object in the response
        start_idx = response_text.find('{')
        end_idx = response_text.rfind('}') + 1
        if start_idx >= 0 and end_idx > start_idx:
            json_str = response_text[start_idx:end_idx]
            result = json.loads(json_str)
            return result
        else:
            # If no JSON found, return a default fail response
            return {
                "score": 0.0,
                "pass": False,
                "reason": "Invalid response format from validator"
            }
    except json.JSONDecodeError:
        # If JSON parsing fails, return a default fail response
        return {
            "score": 0.0,
            "pass": False,
            "reason": "Failed to parse validator response"
        }

In [64]:
# 6.5th Agent Logic Flow: 
def should_continue_to_synthesis(state: ReasoningState) -> str:
    """Determine whether to continue to synthesis or retry analogical transfer based on CRIT score."""
    crit_result = CRIT_Control_Analogical_Transfer(
        state['base_solutions'],
        state['analogical_transfer'], 
        state['abstraction'],
        state['target_domain']
    )

    score = crit_result.get('score', 0)
    passed = crit_result.get('pass', False)
    
    if passed:
        return "synthesis"  # Continue to final synthesis
    else:
        state['transfer_feedback'] = crit_result.get('reason', '')
        return "analogical_transfer"  # Retry transfer with feedback


In [65]:
# 7th Agent: Summarize everything and respond to the question
def synthesis_agent(state: ReasoningState):
    msg = llms.invoke(
        f"Evaluate the proposed analogical solutions. Find the best ones that balances practicality with innovation. Then, provide a detailed, well-structured response that addresses all aspects of the query.\n\n"
        f"Problem: {state['user_query']}\n"
        f"Analogical Solutions: {state['analogical_transfer']}"
        f"In your output, remember to abstract away the analogy itself such that it is focused on responding to the user input."
        f"Also, check if the users are requesting a specific number of possible solutions. Make sure to answer the user's query in full and provide what is requested."
    )
    return {"solution": msg.content}

In [66]:
# Construct the workflow 
workflow = StateGraph(ReasoningState)

workflow.add_node("target", target_domain_agent)
workflow.add_node("landscape", problem_landscape_agent)
workflow.add_node("abstract", abstraction_agent)
workflow.add_node("base", base_domain_agent)
workflow.add_node("base_soln", base_solution_agent)
workflow.add_node("analogy", analogical_transfer_agent)
workflow.add_node("synthesis", synthesis_agent)

workflow.set_entry_point("target")

# Define edges
workflow.add_edge("target", "landscape")
workflow.add_edge("landscape", "abstract")
# workflow.add_edge("abstract", "base")

# Define conditional edges - Use CRIT TO control quality for Abstraction TRIZ
workflow.add_conditional_edges(
    "abstract",
    should_continue_to_base,
    path_map=["base", "abstract"]
)

workflow.add_edge("base", "base_soln")
workflow.add_edge("base_soln", "analogy")
#workflow.add_edge("analogy", "synthesis")

# Define conditional edges - Use CRIT TO control quality for Analogical Transfer
workflow.add_conditional_edges(
    "analogy",
    should_continue_to_synthesis,
    path_map=["synthesis", "analogy"]
)

workflow.set_finish_point("synthesis")

graph = workflow.compile()

### Initialize Different Functions for Prompts

In [71]:
# Basic Output
def basic_output(user_input: str):
    
    prompt = f"""
    {user_input}
    """

    response = llms.invoke(prompt) 

    # Parse and format the response
    formatted_response = parse_solution(response.content)

    return(formatted_response)

In [72]:
# Basic Output with COT
## TO EDIT WITH FULL PROMPT?
def basic_output_COT(user_input: str):
    
    prompt = f"""
    {user_input}
    Think step by step.
    """

    response = llms.invoke(prompt) 

    # Parse and format the response
    formatted_response = parse_solution(response.content)

    return(formatted_response)

In [73]:
# Basic Output with Prompt Engineering
def basic_output_prompt_engin(user_input: str):
    
    prompt = f"""
    {user_input}
    Try to balance practicality with innovation.
    """

    response = llms.invoke(prompt) 

    # Parse and format the response
    formatted_response = parse_solution(response.content)

    return(formatted_response)

### Test prompt 1 - Education

#### User Input

In [74]:
test_q = "Teachers introduced structured group projects using peer evaluations and rotating roles to boost collaboration in science classes. Despite these measures, anonymous student surveys revealed that individual quiz scores post-project dropped compared to solo assignments, signaling diluted accountability. How can educators redesign design a system to ensure measurable individual mastery while preserving the benefits of teamwork?"

#### With Analogical Reasoning

In [ ]:
## Create Analogical Reasoning response
input_state = {"user_query": test_q}

final_state = graph.invoke(input_state)


=== GPT-4.1 Response WITH Analogical Reasoning Pipeline ===

{'user_query': 'Teachers introduced structured group projects using peer evaluations and rotating roles to boost collaboration in science classes. Despite these measures, anonymous student surveys revealed that individual quiz scores post-project dropped compared to solo assignments, signaling diluted accountability. How can educators redesign design a system to ensure measurable individual mastery while preserving the benefits of teamwork?', 'target_domain': 'Assessment Systems Innovation in Collaborative Learning', 'problem_landscape': '[\n  {\n    "Problem": "Lack of Valid and Reliable Assessment Tools for Collaboration",\n    "Description": "Current assessment tools often fail to accurately measure individual contributions and collaborative skills in group learning contexts. This leads to potential unfairness and does not reliably reflect individual or group learning outcomes.",\n    "Context": "K-12 and higher education

In [83]:
# parse_output
import re

def parse_solution(text):
    # Remove Markdown formatting like "**" and "\n"
    clean_text = text.replace("**", "").replace("\\n", "\n")

    # Split into sections by horizontal rules (---)
    sections = re.split(r'-{3,}', clean_text)

    # Create a readable version of each section
    readable_output = []
    for section in sections:
        section = section.strip()
        if not section:
            continue
        # Optional: separate title and body if present
        lines = section.split("\n")
        if len(lines) > 1 and ":" not in lines[0]:
            # First line is likely a heading
            heading = lines[0]
            body = "\n".join(lines[1:])
            readable_output.append(f"\n=== {heading} ===\n{body}")
        else:
            readable_output.append(section)

    return "\n\n".join(readable_output)

In [84]:
# Save all parsed outputs to a text file
with open("sample_output.txt", "w", encoding="utf-8") as f:
    for key in final_state:
        raw_output = str(final_state[key])  # Ensure it's a string
        final_output = parse_solution(raw_output)
        f.write(f"\n### {key} ###\n")
        f.write(final_output)
        f.write("\n\n")

#### Without Analogical Reasoning - GPT 4.1 RAW

In [85]:
## Without analogical reasoning - raw LLM output
result = basic_output(test_q)

print("\n=== GPT-4.1 RAW Response ===\n")
print(result)



=== GPT-4.1 RAW Response ===

This is a thoughtful and common challenge in collaborative learning: preserving the benefits of teamwork (communication, collaboration, deeper thinking) and ensuring individual mastery and accountability.

Here are targeted strategies educators can implement:


=== ### 1. Hybrid Grading Approach ===
- Dual Components: Make the final project grade a combination of group and individual assessments (e.g., 50% group outcome, 50% individual evidence).
- Individual Quizzes or Reflections: Require each student to complete a quiz, written reflection, or oral explanation showing their understanding of the key science concepts behind the project.

### 2. Role-Specific Deliverables
- When rotating roles, assign each student role-specific, individual deliverables. For example, if one student is the "researcher," they must individually summarize sources or findings.
- Collect and grade these outputs separately to verify individual understanding.

### 3. Regular Format

#### Without Analogical Reasoning but with COT

In [86]:
## Without analogical reasoning - raw LLM output
result = basic_output_COT(test_q)

print("\n=== GPT-4.1 Response WITH COT ===\n")
print(result)



=== GPT-4.1 Response WITH COT ===

Absolutely—this is a great, nuanced question about balancing collaboration with individual mastery. Here’s a step-by-step approach educators can use:


=== ### 1. Analyze the Problem ===

- Observation: Teamwork improved via group projects, but individual accountability dropped—reflected by lower post-project quiz scores.
- Challenge: Group work risks “social loafing” (some students do less), hiding who hasn’t mastered the content.


=== ### 2. Clarify Goals ===

- Maintain benefits of teamwork (communication, peer learning, problem-solving).
- Ensure each student achieves and can demonstrate individual mastery.


=== ### 3. Redesign Steps ===

#### Step 1: Incorporate Regular Individual Accountability Within Groups

- Frequent Individual Checks: Mix in *individual* exit tickets, checkpoints, or short quizzes during the project phase—not just at the end. This keeps everyone engaged and signals that individual mastery is always required.
- Randomized 

#### Without Analogical Reasoning but with Basic Prompt Engineering

In [87]:
## Without analogical reasoning - raw LLM output
result = basic_output_prompt_engin(test_q)

print("\n=== GPT-4.1 Response WITH Prompt Engineering ===\n")
print(result)


=== GPT-4.1 Response WITH Prompt Engineering ===

You’ve surfaced a classic tension in group work: collaboration boosts engagement, but can mask—or even reduce—individual learning gains. To strike a balance, consider a redesign that weaves together individual accountability and collaborative skill-building. Here are practical-yet-innovative strategies:


=== ### 1. Hybrid Grading Models ===

- Individual & Group Scores Blend  
  Assign a percentage of the project grade to individual contributions (e.g., quiz/performance, reflection, or portfolio) and another to the group product. For instance, 60% individual, 40% group. Make this transparent to students.  
  *Why it works:* Students have personal “skin in the game,” incentivizing accountability.

- Team Contract with Individual Goals  
  At project kickoff, require each student to articulate learning goals and anticipated contributions. Check-in mid-project on these.  
  *Why it works:* Promotes self-awareness and specific expectation